## experimenting...

In [1]:
from os.path import join

raw = join('data', 'raw')
interim = join('data', 'interim')

from src.training_env import reset_and_populate, populate

reset_and_populate(raw, interim, [400,100,0])

['data\\interim\\train\\n',
 'data\\interim\\validation\\n',
 'data\\interim\\test\\n',
 'data\\interim\\train\\o',
 'data\\interim\\validation\\o',
 'data\\interim\\test\\o',
 'data\\interim\\train\\x',
 'data\\interim\\validation\\x',
 'data\\interim\\test\\x']

## Data generator

In [2]:
import tensorflow as tf
import numpy as np
import pathlib

def create_generator(data_dir, batch_size=32, shuffle_buffer_size=1000):
        def get_label(file_path):
                parts = tf.strings.split(file_path, '\\')
                return parts[-2] == labels

        def decode_img(img):
                img = tf.image.decode_png(img, channels=1)
                img = tf.image.convert_image_dtype(img, tf.float32)
                return tf.image.resize(img, [32, 32])

        def process_path(file_path):
                label = get_label(file_path)
                img = tf.io.read_file(file_path)
                img = decode_img(img)
                return img, label

        data_dir = pathlib.Path(join(interim, data_dir))

        labels = np.array([item.name for item in data_dir.glob('*')])

        autotune = tf.data.experimental.AUTOTUNE

        dataset = (tf.data.Dataset.list_files(str(data_dir/'*/*'))
                .map(process_path, num_parallel_calls=autotune)
                .cache()
                .shuffle(shuffle_buffer_size)
                .repeat()
                .batch(batch_size)
                .prefetch(buffer_size=autotune))
        return dataset

train_generator = create_generator('train', 20)
validation_generator = create_generator('validation', 10)

## Create model

In [3]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adadelta, Adam, RMSprop, SGD

def create_model(hp):
    model = models.Sequential()
    # model.add(layers.Flatten(input_shape=(32, 32, 1)))
    model.add(layers.Conv2D(32, (4, 4), activation='relu', input_shape=(32, 32, 1), data_format='channels_last'))
    model.add(layers.MaxPooling2D((2, 2)))
    def dense(dims):
         model.add(layers.Dense(dims, activation='relu'))
    def conv(dims):
        model.add(layers.Conv2D(dims, (4, 4), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
    layer = {
        'dense': dense,
        'conv': conv
    }
    for i in range(hp.Int('num_layers', 1, 4)):
        dims = 32 # 16 + i * 16 # hp.Int('num_dims_' + str(i), 16, 64, 4)
        print(dims)
        layer['conv'](dims)
        # layer[hp.Choice('layer_' + str(i), list(layer.keys()))](dims)
        # layers.Dropout(hp.Float('dropout_' + str(i), 0, 0.7, 0.1))

    model.add(layers.Flatten())
    model.add(layers.Dense(hp.Int('num_dims_dense', 16, 64, 4), activation='relu'))
    if hp.Boolean('dropout'):
        model.add(layers.Dropout(0.5))
    model.add(layers.Dense(3, 'softmax'))

    optimizers = {
        'adam': Adam(),
        'adadelta': Adadelta(),
        'sgd': SGD(lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01, 0.03]),
            momentum=hp.Float('momentum', 0.6, 1, 0.1),
            nesterov=hp.Boolean('nesterov')),
        'rms': RMSprop(lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01, 0.03]))
    }

    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizers[hp.Choice('optimizer', list(optimizers.keys()))],
        metrics=['acc'])

    return model

In [9]:
def create_model(hp):
    model = models.Sequential()
    model.add(layers.Conv2D(20, (3, 3), activation='relu', input_shape=(32, 32, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    optimizers = {
        'adam': Adam(),
        'adadelta': Adadelta(),
        'sgd': SGD(lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01, 0.03]),
            momentum=hp.Float('momentum', 0.6, 1, 0.1),
            nesterov=hp.Boolean('nesterov')),
        'rms': RMSprop(lr=hp.Choice('learning_rate', [0.001, 0.003, 0.007, 0.01, 0.03]))
    }

    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizers[hp.Choice('optimizer', list(optimizers.keys()))],
        metrics=['acc'])

    return model

In [10]:
from datetime import datetime
from tensorboard.plugins.hparams import api
from kerastuner import RandomSearch
from tensorflow import summary
from tensorflow.keras.callbacks import TensorBoard

class customTuner(RandomSearch):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def run_trial(self, trial, *args, **kwargs):
        callbacks = kwargs.pop('callbacks', [])
        callbacks = self._deepcopy_callbacks(callbacks)
        trial_dir = self.get_trial_dir(trial.trial_id)
        callbacks.append(TensorBoard(trial_dir))
        kwargs['callbacks'] = callbacks
        super().run_trial(trial, *args, **kwargs)

    def on_trial_end(self, trial):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the hparams where the metrics of tensorboard are
        hparam_dir = join(trial_dir, trial.trial_id, 'execution0', 'train')
        hparams = trial.hyperparameters.values
        with summary.create_file_writer(hparam_dir).as_default():
            api.hparams(hparams, trial_id=trial.trial_id)
        print(datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
        print('Remaining Trials: ' + str(self.remaining_trials))
        super().on_trial_end(trial)

    def on_epoch_end(self, trial, model, epoch, logs):
        trial_dir = self.get_trial_dir(trial.trial_id)
        # put the data where the metrics of tensorboard are
        hist_dir = join(trial_dir, trial.trial_id, 'execution0', 'train')
        with summary.create_file_writer(hist_dir).as_default():
            for layer in model.weights:
                summary.histogram(layer.name, data=layer, step=epoch)
        super().on_epoch_end(trial, model, epoch, logs)

In [11]:
from kerastuner import HyperParameters

hp=HyperParameters()
log_dir = 'trial3_logs'
timestamp = datetime.now().strftime("%Y-%m-%dT%H-%M-%S")

tuner = customTuner(
    create_model,
    hyperparameters=hp,
    objective='acc',
    max_trials=2000,
    executions_per_trial=1,
    directory=log_dir,
    project_name=timestamp)

tuner.search_space_summary()

Traceback (most recent call last):
  File "c:\Users\me\devel\deepmech\env\lib\site-packages\tensorflow_core\python\framework\ops.py", line 1610, in _create_c_op
    c_op = c_api.TF_FinishOperation(op_desc)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Negative dimension size caused by subtracting 3 from 2 for 'conv2d_3/Conv2D' (op: 'Conv2D') with input shapes: [?,2,2,128], [3,3,128,128].

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\me\devel\deepmech\env\lib\site-packages\kerastuner\engine\hypermodel.py", line 105, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-9-b4c41ef99bd9>", line 9, in create_model
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  File "c:\Users\me\devel\deepmech\env\lib\site-packages\tensorflow_core\python\training\tracking\base.py", line 457, in _method_wrapper
    result = method(self, *args, **kwargs)
  File "c:\Users\me\devel\deepmech

RuntimeError: Too many failed attempts to build model.

In [0]:
from tensorflow.keras.callbacks import EarlyStopping

callbacks = [ EarlyStopping(monitor='loss', patience=3) ]

tuner.search(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    steps_per_epoch=30,
    validation_steps=30,
    verbose=0,
    callbacks=callbacks)

In [0]:
tuner.results_summary()